## Python Web Crawler

In [17]:
from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse

# Download dir from FTP

Simply Use:
> wget --mirror --continue --no-host-directories --user=username --password=password ftp://hostname/source/path/

# Download Text

http://www.netinstructions.com/how-to-make-a-web-crawler-in-under-50-lines-of-python-code/

In [18]:
# We are going to create a class called LinkParser that inherits some
# methods from HTMLParser which is why it is passed into the definition
class LinkParser(HTMLParser):

    # This is a function that HTMLParser normally has
    # but we are adding some functionality to it
    def handle_starttag(self, tag, attrs):
        # We are looking for the begining of a link. Links normally look
        # like <a href="www.someurl.com"></a>
        if tag == 'a':
            for (key, value) in attrs:
                if key == 'href':
                    # We are grabbing the new URL. We are also adding the
                    # base URL to it. For example:
                    # www.netinstructions.com is the base and
                    # somepage.html is the new URL (a relative URL)
                    #
                    # We combine a relative URL with the base URL to create
                    # an absolute URL like:
                    # www.netinstructions.com/somepage.html
                    newUrl = parse.urljoin(self.baseUrl, value)
                    # And add it to our colection of links:
                    self.links = self.links + [newUrl]

    # This is a new function that we are creating to get links
    # that our spider() function will call
    def getLinks(self, url):
        self.links = []
        # Remember the base URL which will be important when creating
        # absolute URLs
        self.baseUrl = url
        # Use the urlopen function from the standard Python 3 library
        response = urlopen(url)
        # Make sure that we are looking at HTML and not other things that
        # are floating around on the internet (such as
        # JavaScript files, CSS, or .PDFs for example)
        if response.getheader('Content-Type')=='text/html':
            htmlBytes = response.read()
            # Note that feed() handles Strings well, but not bytes
            # (A change from Python 2.x to Python 3.x)
            htmlString = htmlBytes.decode("utf-8")
            self.feed(htmlString)
            return htmlString, self.links
        else:
            return "",[]

In [19]:
from bs4 import BeautifulSoup

def parseHtml_cnfoodsafety(htmlstring):
    soup = BeautifulSoup(htmlstring, "lxml")    
    title = soup.find_all("h1", class_="artical")
    if title == []:
        title = ''
    else:
        title = title[0].get_text()
        
    content = soup.find_all("div", id="artContent")
    if content == []:
        content = ''
    else:
        content = content[0].get_text()
        
    return title, content

In [20]:
from bs4 import BeautifulSoup

def parseHtml_cfsn(htmlstring):
    soup = BeautifulSoup(htmlstring, "lxml")    
    title = soup.find_all("h1")
    if title == []:
        title = ''
    else:
        title = title[0].get_text()
        
    content = soup.find_all("div", id="cont1")
    if content == []:
        content = ''
    else:
        content = content[0].get_text()
        
    return title, content

In [21]:
import pandas as pd

# And finally here is our spider. It takes in an URL, a word to find,
# and the number of pages to search through before giving up
def spider(url, maxPages, pattern):  
    pagesToVisit = [url]
    numberVisited = 0
    pagesVisited = []
    df = pd.DataFrame(columns = ['url', 'date', 'title', 'content'])
    # The main loop. Create a LinkParser and get all the links on the page.
    # Also search the page for the word or string
    # In our getLinks function we return the web page
    # (this is useful for searching for the word)
    # and we return a set of links from that web page
    # (this is useful for where to go next)
    while numberVisited < maxPages and pagesToVisit != []:
        numberVisited = numberVisited +1
        # Start from the beginning of our collection of pages to visit:
        url = pagesToVisit[0]
        pagesToVisit = pagesToVisit[1:]
        #print(pagesToVisit)
        try:
            print(numberVisited, "Visiting:", url)
            parser = LinkParser()
            data, links = parser.getLinks(url)
            if(url.startswith(pattern)):
                date = url[len(pattern) - 2: len(pattern) - 2 + 9 + 1]
                title = ''
                content = ''
                if 'cnfoodsafety' in url:
                    title, content = parseHtml_cnfoodsafety(data)
                elif 'cfsn' in url:
                    title, content = parseHtml_cfsn(data)
                if content != '':
                    df.loc[-1] = [url, date, title, content] # add a row
                    df.index = df.index + 1
            #print(title, content)
                      
            # Add the pages that we visited to the end of our collection of pages to visit:
            # pagesToVisit = pagesToVisit + links   
            for link in links:
                link = link.replace('#SOHUCS','').replace('#comment','')
                if(link.startswith(pattern) and link not in pagesVisited):
                    # pagesToVisit.extend(links)
                    pagesVisited.append(link)
                    pagesToVisit.append(link)
            print(" **Success!**")
        except:
            print(" **Failed!**")
        
    return df

## 中国食品安全网 - 曝光专栏

In [15]:
from datetime import datetime
fname = datetime.now().strftime('%Y%m%d%H%M%S')

df = spider('http://www.cfsn.cn/news/node_4534.htm', 1000, 'http://www.cfsn.cn/news/content/20')
for url in ['http://www.cfsn.cn/news/node_4534_2.htm', 'http://www.cfsn.cn/news/node_4534_3.htm', 'http://www.cfsn.cn/news/node_4534_4.htm', 'http://www.cfsn.cn/news/node_4534_5.htm']:
    df_tmp = spider(url, 1000, 'http://www.cfsn.cn/news/content/20')
    df = pd.concat([df, df_tmp], ignore_index=True)

with pd.HDFStore(fname + '.h5',  mode='w') as store:
    store.append('df', df, data_columns= df.columns, format='table')

1 Visiting: http://www.cfsn.cn/news/node_4534.htm
 **Success!**
2 Visiting: http://www.cfsn.cn/news/content/2018-05/04/content_290905.htm
 **Success!**
3 Visiting: http://www.cfsn.cn/news/content/2018-04/25/content_290806.htm
 **Success!**
4 Visiting: http://www.cfsn.cn/news/content/2018-04/25/content_290805.htm
 **Success!**
5 Visiting: http://www.cfsn.cn/news/content/2018-04/23/content_290723.htm
 **Success!**
6 Visiting: http://www.cfsn.cn/news/content/2018-01/26/content_289588.htm
 **Success!**
7 Visiting: http://www.cfsn.cn/news/content/2018-01/26/content_289587.htm
 **Success!**
8 Visiting: http://www.cfsn.cn/news/content/2018-01/26/content_289586.htm
 **Success!**
9 Visiting: http://www.cfsn.cn/news/content/2018-01/26/content_289585.htm
 **Success!**
10 Visiting: http://www.cfsn.cn/news/content/2018-01/25/content_289573.htm
 **Success!**
11 Visiting: http://www.cfsn.cn/news/content/2018-01/25/content_289572.htm
 **Success!**
12 Visiting: http://www.cfsn.cn/news/content/2018-01/2

 **Success!**
43 Visiting: http://www.cfsn.cn/news/content/2016-09/20/content_281250.htm
 **Success!**
44 Visiting: http://www.cfsn.cn/news/content/2016-09/07/content_281116.htm
 **Success!**
45 Visiting: http://www.cfsn.cn/news/content/2016-09/02/content_281022.htm
 **Success!**
46 Visiting: http://www.cfsn.cn/news/content/2016-08/26/content_280884.htm
 **Success!**
47 Visiting: http://www.cfsn.cn/news/content/2016-08/26/content_280883.htm
 **Success!**
48 Visiting: http://www.cfsn.cn/news/content/2016-08/23/content_280801.htm
 **Success!**
49 Visiting: http://www.cfsn.cn/news/content/2016-08/23/content_280798.htm
 **Success!**
50 Visiting: http://www.cfsn.cn/news/content/2016-08/23/content_280741.htm
 **Success!**
51 Visiting: http://www.cfsn.cn/news/content/2016-08/19/content_280678.htm
 **Success!**
1 Visiting: http://www.cfsn.cn/news/node_4534_3.htm
 **Success!**
2 Visiting: http://www.cfsn.cn/news/content/2016-08/19/content_280677.htm
 **Success!**
3 Visiting: http://www.cfsn.cn/

 **Success!**
38 Visiting: http://www.cfsn.cn/news/content/2013-05/31/content_127187.htm
 **Success!**
39 Visiting: http://www.cfsn.cn/news/content/2013-05/31/content_127186.htm
 **Success!**
40 Visiting: http://www.cfsn.cn/news/content/2013-05/28/content_125318.htm
 **Success!**
41 Visiting: http://www.cfsn.cn/news/content/2013-05/16/content_121074.htm
 **Success!**
42 Visiting: http://www.cfsn.cn/news/content/2013-04/25/content_112830.htm
 **Success!**
43 Visiting: http://www.cfsn.cn/news/content/2013-04/18/content_109984.htm
 **Success!**
44 Visiting: http://www.cfsn.cn/news/content/2013-09/05/content_149451.htm
 **Success!**
45 Visiting: http://www.cfsn.cn/news/content/2013-10/10/content_153886.htm
 **Success!**
46 Visiting: http://www.cfsn.cn/news/content/2013-08/16/content_146623.htm
 **Success!**
47 Visiting: http://www.cfsn.cn/news/content/2012-12/20/content_79489.htm
 **Success!**
48 Visiting: http://www.cfsn.cn/news/content/2011-10/10/content_14453.htm
 **Success!**
49 Visiti

C:\Users\penguin\Anaconda3\Lib\site-packages\tables\leaf.py:386: PerformanceWarning: The Leaf ``/df/_i_table/content/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


## 中国食品安全网 - 专家解读专栏

In [22]:
from datetime import datetime
fname = datetime.now().strftime('%Y%m%d%H%M%S')

df = spider('http://www.cfsn.cn/zhuanti/node_5253.htm', 1000, 'http://www.cfsn.cn/zhuanti/20')

with pd.HDFStore(fname + '.h5',  mode='w') as store:
    store.append('df', df, data_columns= df.columns, format='table')

1 Visiting: http://www.cfsn.cn/zhuanti/node_5253.htm
 **Success!**
2 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84203.htm
 **Success!**
3 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84204.htm
 **Success!**
4 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84211.htm
 **Success!**
5 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84212.htm
 **Success!**
6 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84217.htm
 **Success!**
7 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84218.htm
 **Success!**
8 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84221.htm
 **Success!**
9 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84222.htm
 **Success!**
10 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84225.htm
 **Success!**
11 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84226.htm
 **Success!**
12 Visiting: http://www.cfsn.cn/zhuanti/2013-01/06/content_84229.htm
 **Success!**
13 Visiting: http://www.cfsn

C:\Users\penguin\Anaconda3\Lib\site-packages\tables\leaf.py:386: PerformanceWarning: The Leaf ``/df/_i_table/content/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


In [23]:
df

,url,date,title,content
23,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n “螺旋藻铅超标”热点概况\r\n,\n\n2012年3月，据中国之声《新闻晚高峰》报道，包括绿A公司在内的6大螺旋藻品牌被媒体...
22,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n 专家观点\r\n,\n\n北京大学工学院食品与生物资源工程研究所所长 陈峰\n螺旋藻是最受欢迎的藻类保健食品之...
21,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n “明胶门”热点概况\r\n,\n\n2012年4月9日，央视某主持人发微博称：“来自调查记者短信：不要再吃老酸奶（固体形...
20,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n 专家观点\r\n,\n\n中国食品发酵工业研究院前院长国家食品药品监督管理局特邀专家 孙熲\n食用明胶的标准中...
19,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n “立顿农药门”热点概况\r\n,\n\n“绿色和平组织”于2012年3月对全球最大的茶叶品牌——“立顿”牌袋泡茶叶进行了抽样...
18,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n 专家观点\r\n,\n\n江南大学食品学院院长 陈卫\n根据我国国家标准GB/T24690-2009“袋泡茶”...
17,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n “可口可乐含氯门”热点概况\r\n,\n\n2012年4月16日，有媒体报道披露，可口可乐山西饮料公司因管道改造，致使消毒用的含...
16,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n 专家观点\r\n,\n\n中国工程院院士，\n北京工商大学副校长，CIFST副理事长 孙宝国\n这是一起典型的...
15,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n “古越龙山黄酒含有致癌物”热点概况\r\n,\n\n2012年6月15日，香港消费者委员会称浙江绍兴的两款黄酒中含有2A类可疑致癌物氨基...
14,http://www.cfsn.cn/zhuanti/2013-01/06/content_...,2013-01/06,\r\n 专家观点\r\n,\n\n中国食品发酵工业研究院教授级高工 熊正河\n氨基甲酸乙酯（简称，EC）是广泛存在于发...


## 食安中国

In [12]:
from datetime import datetime
fname = datetime.now().strftime('%Y%m%d%H%M%S')

df = spider('http://www.cnfoodsafety.com/news/spaq/', 100000, 'http://www.cnfoodsafety.com/20')

with pd.HDFStore(fname + '.h5',  mode='w') as store:
    store.append('df', df, data_columns= df.columns, format='table')

1 Visiting: http://www.cnfoodsafety.com/news/spaq/
 **Success!**
2 Visiting: http://www.cnfoodsafety.com/2018/0424/26918.html
 **Success!**
3 Visiting: http://www.cnfoodsafety.com/2018/0424/26917.html
 **Success!**
4 Visiting: http://www.cnfoodsafety.com/2018/0424/26915.html
 **Success!**
5 Visiting: http://www.cnfoodsafety.com/2018/0424/26914.html
 **Success!**
6 Visiting: http://www.cnfoodsafety.com/2018/0424/26913.html
 **Success!**
7 Visiting: http://www.cnfoodsafety.com/2018/0424/26912.html
 **Success!**
8 Visiting: http://www.cnfoodsafety.com/2018/0424/26885.html
 **Success!**
9 Visiting: http://www.cnfoodsafety.com/2018/0424/26884.html
 **Success!**
10 Visiting: http://www.cnfoodsafety.com/2018/0424/26883.html
 **Success!**
11 Visiting: http://www.cnfoodsafety.com/2018/0424/26882.html
 **Success!**
12 Visiting: http://www.cnfoodsafety.com/2018/0424/26881.html
 **Success!**
13 Visiting: http://www.cnfoodsafety.com/2018/0420/26852.html
 **Success!**
14 Visiting: http://www.cnfoods

 **Success!**
110 Visiting: http://www.cnfoodsafety.com/2014/1112/6177.html
 **Success!**
111 Visiting: http://www.cnfoodsafety.com/2016/1028/20996.html
 **Success!**
112 Visiting: http://www.cnfoodsafety.com/2017/1106/25191.html
 **Success!**
113 Visiting: http://www.cnfoodsafety.com/2014/0917/4029.html
 **Success!**
114 Visiting: http://www.cnfoodsafety.com/2014/0616/137.html
 **Success!**
115 Visiting: http://www.cnfoodsafety.com/2015/0902/14470.html
 **Success!**
116 Visiting: http://www.cnfoodsafety.com/2015/1202/16036.html
 **Success!**
117 Visiting: http://www.cnfoodsafety.com/2016/0727/19629.html
 **Success!**
118 Visiting: http://www.cnfoodsafety.com/2017/1110/25241.html
 **Success!**
119 Visiting: http://www.cnfoodsafety.com/2014/0616/165.html
 **Success!**
120 Visiting: http://www.cnfoodsafety.com/2014/0626/884.html
 **Success!**
121 Visiting: http://www.cnfoodsafety.com/2015/0623/12501.html
 **Success!**
122 Visiting: http://www.cnfoodsafety.com/2016/0229/17456.html
 **Succ

 **Success!**
218 Visiting: http://www.cnfoodsafety.com/2015/1109/15613.html
 **Success!**
219 Visiting: http://www.cnfoodsafety.com/2016/0324/17757.html
 **Success!**
220 Visiting: http://www.cnfoodsafety.com/2015/1122/15855.html
 **Success!**
221 Visiting: http://www.cnfoodsafety.com/2015/1122/15854.html
 **Success!**
222 Visiting: http://www.cnfoodsafety.com/2014/0625/824.html
 **Success!**
223 Visiting: http://www.cnfoodsafety.com/2016/0121/16908.html
 **Success!**
224 Visiting: http://www.cnfoodsafety.com/2016/0309/17577.html
 **Success!**
225 Visiting: http://www.cnfoodsafety.com/2014/0616/13.html
 **Success!**
226 Visiting: http://www.cnfoodsafety.com/2014/0721/1346.html
 **Success!**
227 Visiting: http://www.cnfoodsafety.com/2016/1208/21702.html
 **Success!**
228 Visiting: http://www.cnfoodsafety.com/2017/0925/24751.html
 **Success!**
229 Visiting: http://www.cnfoodsafety.com/2017/0919/24649.html
 **Failed!**
230 Visiting: http://www.cnfoodsafety.com/2015/0610/12245.html
 **Suc

 **Success!**
326 Visiting: http://www.cnfoodsafety.com/2015/0511/11099.html
 **Success!**
327 Visiting: http://www.cnfoodsafety.com/2016/1024/20925.html
 **Success!**
328 Visiting: http://www.cnfoodsafety.com/2016/1024/20930.html
 **Success!**
329 Visiting: http://www.cnfoodsafety.com/2014/0930/4697.html
 **Success!**
330 Visiting: http://www.cnfoodsafety.com/2015/0304/9316.html
 **Success!**
331 Visiting: http://www.cnfoodsafety.com/2016/1024/20897.html
 **Success!**
332 Visiting: http://www.cnfoodsafety.com/2016/1021/20876.html
 **Success!**
333 Visiting: http://www.cnfoodsafety.com/2016/1020/20867.html
 **Success!**
334 Visiting: http://www.cnfoodsafety.com/2016/1019/20862.html
 **Success!**
335 Visiting: http://www.cnfoodsafety.com/2016/1019/20837.html
 **Success!**
336 Visiting: http://www.cnfoodsafety.com/2014/0617/223.html
 **Success!**
337 Visiting: http://www.cnfoodsafety.com/2014/1014/5005.html
 **Success!**
338 Visiting: http://www.cnfoodsafety.com/2015/0730/13567.html
 **S

 **Success!**
434 Visiting: http://www.cnfoodsafety.com/2017/0511/23268.html
 **Success!**
435 Visiting: http://www.cnfoodsafety.com/2015/0821/14168.html
 **Success!**
436 Visiting: http://www.cnfoodsafety.com/2017/1218/25668.html
 **Success!**
437 Visiting: http://www.cnfoodsafety.com/2016/0929/20623.html
 **Success!**
438 Visiting: http://www.cnfoodsafety.com/2016/0531/18896.html
 **Success!**
439 Visiting: http://www.cnfoodsafety.com/2016/0323/17756.html
 **Success!**
440 Visiting: http://www.cnfoodsafety.com/2016/0415/18106.html
 **Success!**
441 Visiting: http://www.cnfoodsafety.com/2016/0325/17774.html
 **Success!**
442 Visiting: http://www.cnfoodsafety.com/2016/0424/18209.html
 **Success!**
443 Visiting: http://www.cnfoodsafety.com/2017/0224/22510.html
 **Success!**
444 Visiting: http://www.cnfoodsafety.com/2016/0822/19991.html
 **Success!**
445 Visiting: http://www.cnfoodsafety.com/2016/0530/18883.html
 **Success!**
446 Visiting: http://www.cnfoodsafety.com/2014/1216/7575.html


...

 **Success!**
4059 Visiting: http://www.cnfoodsafety.com/2014/0916/3927.html
 **Success!**
4060 Visiting: http://www.cnfoodsafety.com/2014/0925/4318.html
 **Success!**
4061 Visiting: http://www.cnfoodsafety.com/2015/1118/15753.html
 **Success!**
4062 Visiting: http://www.cnfoodsafety.com/2015/0410/10378.html
 **Success!**
4063 Visiting: http://www.cnfoodsafety.com/2015/0319/9724.html
 **Success!**
4064 Visiting: http://www.cnfoodsafety.com/2015/0427/10777.html
 **Success!**
4065 Visiting: http://www.cnfoodsafety.com/2015/0703/12813.html
 **Success!**
4066 Visiting: http://www.cnfoodsafety.com/2015/0313/9515.html
 **Success!**
4067 Visiting: http://www.cnfoodsafety.com/2014/1107/5977.html
 **Success!**
4068 Visiting: http://www.cnfoodsafety.com/2015/0629/12681.html
 **Success!**
4069 Visiting: http://www.cnfoodsafety.com/2015/0130/8789.html
 **Success!**
4070 Visiting: http://www.cnfoodsafety.com/2015/0505/10951.html
 **Success!**
4071 Visiting: http://www.cnfoodsafety.com/2015/0713/130

 **Success!**
4165 Visiting: http://www.cnfoodsafety.com/2015/0826/14330.html
 **Success!**
4166 Visiting: http://www.cnfoodsafety.com/2015/0527/11686.html
 **Success!**
4167 Visiting: http://www.cnfoodsafety.com/2014/1104/5825.html
 **Success!**
4168 Visiting: http://www.cnfoodsafety.com/2015/1207/16094.html
 **Success!**
4169 Visiting: http://www.cnfoodsafety.com/2015/0108/8253.html
 **Success!**
4170 Visiting: http://www.cnfoodsafety.com/2014/1209/7292.html
 **Success!**
4171 Visiting: http://www.cnfoodsafety.com/2015/1021/15264.html
 **Success!**
4172 Visiting: http://www.cnfoodsafety.com/2015/0724/13339.html
 **Success!**
4173 Visiting: http://www.cnfoodsafety.com/2015/1124/15898.html
 **Success!**
4174 Visiting: http://www.cnfoodsafety.com/2015/1109/15598.html
 **Success!**
4175 Visiting: http://www.cnfoodsafety.com/2015/0707/12869.html
 **Success!**
4176 Visiting: http://www.cnfoodsafety.com/2015/0714/13037.html
 **Success!**
4177 Visiting: http://www.cnfoodsafety.com/2015/0119/

 **Success!**
4273 Visiting: http://www.cnfoodsafety.com/2014/1011/4978.html
 **Success!**
4274 Visiting: http://www.cnfoodsafety.com/2015/0729/13535.html
 **Success!**
4275 Visiting: http://www.cnfoodsafety.com/2014/0830/3065.html
 **Success!**
4276 Visiting: http://www.cnfoodsafety.com/2014/0901/3117.html
 **Success!**
4277 Visiting: http://www.cnfoodsafety.com/2016/0112/16697.html
 **Success!**
4278 Visiting: http://www.cnfoodsafety.com/2016/0115/16784.html
 **Success!**
4279 Visiting: http://www.cnfoodsafety.com/2014/0825/2914.html
 **Success!**
4280 Visiting: http://www.cnfoodsafety.com/2014/0901/3121.html
 **Success!**
4281 Visiting: http://www.cnfoodsafety.com/2014/1009/4851.html
 **Success!**
4282 Visiting: http://www.cnfoodsafety.com/2014/0903/3307.html
 **Success!**
4283 Visiting: http://www.cnfoodsafety.com/2015/0820/14142.html
 **Success!**
4284 Visiting: http://www.cnfoodsafety.com/2014/1121/6571.html
 **Success!**
4285 Visiting: http://www.cnfoodsafety.com/2015/0428/10820

 **Success!**
4379 Visiting: http://www.cnfoodsafety.com/2014/0731/1751.html
 **Success!**
4380 Visiting: http://www.cnfoodsafety.com/2014/0808/2212.html
 **Success!**
4381 Visiting: http://www.cnfoodsafety.com/2014/0915/3806.html
 **Success!**
4382 Visiting: http://www.cnfoodsafety.com/2014/1104/5820.html
 **Success!**
4383 Visiting: http://www.cnfoodsafety.com/2015/0515/11286.html
 **Success!**
4384 Visiting: http://www.cnfoodsafety.com/2014/0626/871.html
 **Success!**
4385 Visiting: http://www.cnfoodsafety.com/2014/0818/2601.html
 **Success!**
4386 Visiting: http://www.cnfoodsafety.com/2014/0903/3302.html
 **Success!**
4387 Visiting: http://www.cnfoodsafety.com/2015/0817/14046.html
 **Success!**
4388 Visiting: http://www.cnfoodsafety.com/2015/0824/14255.html
 **Success!**
4389 Visiting: http://www.cnfoodsafety.com/2015/0810/13836.html
 **Success!**
4390 Visiting: http://www.cnfoodsafety.com/2015/0824/14236.html
 **Success!**
4391 Visiting: http://www.cnfoodsafety.com/2015/1013/15136

C:\Users\penguin\Anaconda3\Lib\site-packages\tables\leaf.py:386: PerformanceWarning: The Leaf ``/df/_i_table/content/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


### 读取数据

In [1]:
import pandas as pd

with pd.HDFStore('20180504234042.h5',  mode='r') as newstore:
    df_restored = newstore.select('df')

C:\Users\penguin\Anaconda3\Lib\site-packages\tables\leaf.py:386: PerformanceWarning: The Leaf ``/df/_i_table/content/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


### PYNIPIR(ICTCLAS - Institute of Computing Technology，Chinese Lexical Analysis System)

> pip install pynlpir   
> pynlpir update

In [19]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器

from ctypes import c_char_p
pynlpir.nlpir.AddUserWord(c_char_p("食品安全".encode()))

for item in df_restored.values[:5]:
    print(item[2], '\t', pynlpir.segment(item[2], pos_english=False, pos_tagging=True), '\n', pynlpir.get_key_words(item[3]), '\n\n')
    
pynlpir.close()

dir(pynlpir)

山东省高级人民法院公布四起典型案例 	 [('山东省', '名词'), ('高级', '形容词'), ('人民法院', '名词'), ('公布', '动词'), ('四', '数词'), ('起', '量词'), ('典型', '名词'), ('案例', '名词')] 
 ['姜宏战', '张占', '被告人', '销售', '生产', '有期徒刑', '食品', '工业明胶', '刘明光', '人民群众', '煮胶', '非法经营罪', '祁志华', '判处被告人', '原料', '审判', '工作', '法院', '食品罪', '食品安全', '国家规定', '用于', '犯罪', '使用', '情节', '王庆荣', '经营', '人民', '危害', '武城县', '工业', '国家', '添加', '硫酸', '严重', '上诉', '社会', '刑罚', '职能', '参与', '餐桌', '提供', '违反', '禁止', '明知', '活动', '收购', '组织', '蒸锅', '负责'] 


青岛公布危害食药安全犯罪典型案例 	 [('青岛', '名词'), ('公布', '动词'), ('危害', '动词'), ('食', '动词'), ('药', '名词'), ('安全', '形容词'), ('犯罪', '动词'), ('典型', '名词'), ('案例', '名词')] 
 ['销售', '高某', '赵某', '加工', '牛肚', '生产', '使用', '食品', '对外销售', '进行', '甲醛', '成分', '屠宰', '大葱', '国家', '经营', '盐产品', '非食品原料', '明知', '判处有期徒刑', '松香', '白菜', '收购', '添加', '生猪', '含有', '保健食品', '原料', '非食用', '工业', '销售金额', '胡某', '工业松香', '屠宰生猪', '使用国家', '食用', '扣押', '禁止', '粮食', '食品罪', '巴西', '构成', '有期徒刑', '产品', '制品', '有毒', '检测', '安排', '物质', '火碱'] 


食品微生物快速检测技术研究 	 [('食品', '名词'), ('微生物', '名词'), ('快速', '区别词'), 

['ENCODING',
 'ENCODING_ERRORS',
 'LicenseError',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_attempt_to_raise_license_error',
 '_decode',
 '_encode',
 '_get_pos_name',
 '_to_float',
 'close',
 'dt',
 'fopen',
 'get_key_words',
 'is_python3',
 'logger',
 'logging',
 'nlpir',
 'open',
 'os',
 'pos_map',
 'segment',
 'sys',
 'unicode',
 'unicode_literals']

### SnowNLP

In [54]:
from snownlp import SnowNLP

for item in df_restored.values[:10]:
    s = SnowNLP(item[3])
    print(item[2], '\t', s.keywords(5), '\n', s.summary(3), '\n\n')

山东省高级人民法院公布四起典型案例 	 ['被告人', '销售', '生产', '食品', '人'] 
 ['以生产、销售不符合安全标准的食品罪分别判处被告人宋振有期徒刑三年', '以生产、销售有毒、有害食品罪分别判处被告人王庆荣有期徒刑十年', '以生产、销售有毒、有害食品罪、非法经营罪分别判处被告人姜宏战有期徒刑十四年、一年'] 


青岛公布危害食药安全犯罪典型案例 	 ['销售', '某某', '中', '食品', '案例'] 
 ['马某某犯生产、销售有毒、有害食品罪', '董某某在生产、销售的食品中掺入有毒、有害的非食品原料', '迟某某在生产、销售的食品中掺入有毒、有害的非法食品原料'] 


食品微生物快速检测技术研究 	 ['检测', '技术', '方法', '中', '扩'] 
 ['核酸依赖性扩增检测技术核酸依赖性扩增检测技术（NASBA）是一项以RNA模板进行等温核酸扩增并能实时监测结果的检测方法', '普通PCR PCR技术被广泛用于各类食品中致病菌及毒素的检测', '非PCR核酸检测技术'] 


检测实验室申请认可的关键环节 	 ['实验室', '认可', '检测', '质量', '文件'] 
 ['基本认可准则即CNAS-CL01《检测和校准实验室能力认可准则》', '确定申请认可的检测对象、检测项目、检测方法', '按照CNAS认可准则的要求建立管理体系'] 


甜酒酿的制作 	 ['中', '糯米', '酒', '酿', '发酵'] 
 ['另外甜酒酿中这些氨基酸含量虽然不是很高', '发酵好的甜酒酿从生化培养箱中拿出', '酒酿中的脂肪含量很低'] 


液相串联质谱法测定饮用水中甲醛 	 ['甲醛', '质', '溶液', '谱', '离子'] 
 ['本实验采用乙腈+0.1%甲酸水溶液体系作为流动相', '衍生后液相串联质谱仪测定', '液相串联质谱法测定饮用水中的甲醛'] 


食品安全分析方法的研究 	 ['分析', '食品', '法', '测定', '高'] 
 ['荧光分析法的好处是灵敏度高', '荧光分光光度法方法的选择性较分光光度法有所提高', '在计算机技术和联用技术（如质谱核磁共振波谱）的发展下,有着高灵敏度、高质量、高效率特点的高效液相技术的应用将越来越广泛'] 


药品微生物限度检查和食品微生物检验的

In [7]:
from snownlp import SnowNLP

s = SnowNLP(u'这个东西真心很赞')

print(s.words)         # [u'这个', u'东西', u'真心',
                #  u'很', u'赞']

print(s.sentiments)    # 0.9769663402895832 positive的概率

print(s.pinyin)        # [u'zhe', u'ge', u'dong', u'xi',
                #  u'zhen', u'xin', u'hen', u'zan']

['这个', '东西', '真心', '很', '赞']
0.9769551298267365
['zhe', 'ge', 'dong', 'xi', 'zhen', 'xin', 'hen', 'zan']


In [15]:
for tag in s.tags:
    print(tag)

('这个', 'r')
('东西', 'n')
('真心', 'd')
('很', 'd')
('赞', 'Vg')


In [4]:
s = SnowNLP(u'「繁體字」「繁體中文」的叫法在臺灣亦很常見。')

s.han           # u'「繁体字」「繁体中文」的叫法
                # 在台湾亦很常见。'

'「繁体字」「繁体中文」的叫法在台湾亦很常见。'

In [19]:
text = u'''
自然语言处理是计算机科学领域与人工智能领域中的一个重要方向。
它研究能实现人与计算机之间用自然语言进行有效通信的各种理论和方法。
自然语言处理是一门融语言学、计算机科学、数学于一体的科学。
因此，这一领域的研究将涉及自然语言，即人们日常使用的语言，
所以它与语言学的研究有着密切的联系，但又有重要的区别。
自然语言处理并不是一般地研究自然语言，
而在于研制能有效地实现自然语言通信的计算机系统，
特别是其中的软件系统。因而它是计算机科学的一部分。
'''

s = SnowNLP(text)

print(s.keywords(3))

print(s.summary(2))

print(s.sentences)

['语言', '自然', '计算机']
['因而它是计算机科学的一部分', '自然语言处理是计算机科学领域与人工智能领域中的一个重要方向']
['自然语言处理是计算机科学领域与人工智能领域中的一个重要方向', '它研究能实现人与计算机之间用自然语言进行有效通信的各种理论和方法', '自然语言处理是一门融语言学、计算机科学、数学于一体的科学', '因此', '这一领域的研究将涉及自然语言', '即人们日常使用的语言', '所以它与语言学的研究有着密切的联系', '但又有重要的区别', '自然语言处理并不是一般地研究自然语言', '而在于研制能有效地实现自然语言通信的计算机系统', '特别是其中的软件系统', '因而它是计算机科学的一部分']
